In [24]:
import os, re
import arcpy
from arcpy.sa import *

# ---- inputs ----
gdb_points = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb"
points_fc  = "TrainingPts"
rasters_folder = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\RASTERS_ALIGNED"

arcpy.CheckOutExtension("Spatial")
arcpy.env.workspace = gdb_points
arcpy.env.overwriteOutput = True

in_points = os.path.join(gdb_points, points_fc)

# optional snap
snap = os.path.join(rasters_folder, "dem10.tif")
if arcpy.Exists(snap):
    arcpy.env.snapRaster = snap

# --- helpers ---
def sanitize_field(stem, used):
    s = re.sub(r"[^A-Za-z0-9_]", "_", os.path.splitext(stem)[0])
    if not s or s[0].isdigit():
        s = "f_" + s
    s = s[:30]
    base = s; i = 1
    while s.lower() in used:
        s = (base[:27] + f"_{i}")[:30]; i += 1
    used.add(s.lower()); return s

def is_valid_for_extract(path):
    """Try a few lightweight checks; return True if ok."""
    try:
        if not arcpy.Exists(path): return False
        arcpy.management.GetRasterProperties(path, "BANDCOUNT")
        return True
    except Exception:
        return False

def ensure_extract_ready(path, pixel_type):
    """
    If raster is invalid for ExtractMultiValuesToPoints, try to re-cast it
    to a clean TIFF of the requested pixel_type and return the new path.
    """
    if is_valid_for_extract(path):
        return path
    # repair
    tmp_dir = os.path.join(rasters_folder, "_extract_tmp")
    os.makedirs(tmp_dir, exist_ok=True)
    base = os.path.splitext(os.path.basename(path))[0]
    suffix = "_u8" if "8_BIT" in pixel_type else "_f32"
    out_path = os.path.join(tmp_dir, base + suffix + ".tif")
    if arcpy.Exists(out_path):
        arcpy.management.Delete(out_path)
    arcpy.management.CopyRaster(
        in_raster=path,
        out_rasterdataset=out_path,
        pixel_type=pixel_type,
        format="TIFF"
    )
    # stats help some GP tools accept the raster
    try:
        arcpy.management.CalculateStatistics(out_path)
    except Exception:
        pass
    return out_path if is_valid_for_extract(out_path) else None

# collect rasters
all_tifs = [f for f in os.listdir(rasters_folder) if f.lower().endswith(".tif")]
# exclude your model outputs if any live here
exclude = {"prospectivity", "targets", "binary"}
all_tifs = [f for f in all_tifs if not any(tok in f.lower() for tok in exclude)]

lith_paths = []
cont_paths = []
for f in all_tifs:
    p = os.path.join(rasters_folder, f)
    if f.lower().startswith("lith"):
        lith_paths.append(p)
    else:
        cont_paths.append(p)

# build value tables with auto-repair
used_names = set()

vt_cont = arcpy.ValueTable(2)
for p in cont_paths:
    fixed = ensure_extract_ready(p, "32_BIT_FLOAT")
    if not fixed:
        print(f"[SKIP] Could not repair {os.path.basename(p)} for extraction.")
        continue
    fld = sanitize_field(os.path.basename(p), used_names)
    vt_cont.addRow(f"{fixed} {fld}")

vt_lith = arcpy.ValueTable(2)
for p in lith_paths:
    fixed = ensure_extract_ready(p, "8_BIT_UNSIGNED")
    if not fixed:
        print(f"[SKIP] Could not repair {os.path.basename(p)} for extraction.")
        continue
    fld = sanitize_field(os.path.basename(p), used_names)
    vt_lith.addRow(f"{fixed} {fld}")

print(f"[INFO] Continuous (bilinear) rows: {vt_cont.rowCount}")
print(f"[INFO] Lithology  (nearest)  rows: {vt_lith.rowCount}")

# run extraction
if vt_cont.rowCount > 0:
    print("[RUN] ExtractMultiValuesToPoints (BILINEAR)")
    arcpy.sa.ExtractMultiValuesToPoints(in_points, vt_cont, "BILINEAR")

if vt_lith.rowCount > 0:
    print("[RUN] ExtractMultiValuesToPoints (NEAREST)")
    arcpy.sa.ExtractMultiValuesToPoints(in_points, vt_lith, "NONE")

print("[OK] Training point re-extraction complete.")


[INFO] Continuous (bilinear) rows: 26
[INFO] Lithology  (nearest)  rows: 21
[RUN] ExtractMultiValuesToPoints (BILINEAR)
[RUN] ExtractMultiValuesToPoints (NEAREST)
[OK] Training point re-extraction complete.
